## Unsloth installation and importing libraries

In [ ]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install "torch==2.4.0" "xformers==0.0.27.post2" triton torchvision torchaudio
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import torch
from unsloth import PatchDPOTrainer
from transformers import TrainingArguments
from trl import DPOTrainer
from unsloth import FastLanguageModel
from datasets import load_dataset
PatchDPOTrainer()

## Loading dataset

In [ ]:
# Load the dataset from the specified Hugging Face repository and split it into the 'train' set
dataset = load_dataset("Chirag4579/security-dpo-clean", split='train')

# Rename the 'text' column to 'chosen' for clarity and to align with the expected column naming
dataset = dataset.rename_column('text', 'chosen')

# Rename the 'rejected_text' column to 'rejected' for consistency with the 'chosen' column
dataset = dataset.rename_column('rejected_text', 'rejected')

In [ ]:
dataset

In [ ]:
# In the example we are taking a small sample of data as it will take 1.5 to 2 hours to finetune

# Split the dataset into training and testing sets, with 40% of the data allocated to the test set
dataset = dataset.train_test_split(test_size=0.4)

# Select the first 6000 samples from the training set for further processing
dataset['train'] = dataset['train'].select(range(6000))

# Select the first 2000 samples from the testing set for further processing
dataset['test'] = dataset['test'].select(range(2000))

In [ ]:
dataset

## Loading model and its configurations

In [ ]:
# Define the maximum sequence length for the model to process
max_seq_length = 4096

# Specify the data type for the model (set to None for default behavior)
dtype = None

# Enable 4-bit quantization to reduce memory usage; set to False if higher precision is required
load_in_4bit = True 

# Load the pretrained language model and tokenizer using the specified parameters
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.3-bnb-4bit",  # Model name from the Hugging Face hub
    max_seq_length=max_seq_length,  # Maximum token limit for input sequences
    dtype=dtype,  # Data type for model parameters
    load_in_4bit=load_in_4bit,  # Flag for enabling 4-bit quantization
)

In [ ]:
# Apply PEFT (Parameter-Efficient Fine-Tuning) to the quantized model using LoRA
model = FastLanguageModel.get_peft_model(
    model,  # The pre-trained and potentially quantized model

    # The rank of the low-rank decomposition matrices for LoRA. Higher values allow
    # the model to capture more task-specific information (common values: 8, 16, 32, 64, 128).
    r=64,  

    # Specify the target modules where LoRA should be applied, such as key, value, 
    # query projections, and other attention-related layers.
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention projection layers
        "gate_proj", "up_proj", "down_proj",    # Feed-forward network layers
    ],

    # LoRA-specific hyperparameters:
    lora_alpha=64,  # Scaling factor for the LoRA update
    lora_dropout=0,  # Dropout for LoRA layers; set to 0 for no dropout
    bias="none",  # Bias handling strategy; options are "none", "all", or "lora_only"

    # Enable gradient checkpointing for memory efficiency during training.
    # "unsloth" is a custom value, likely to support longer context lengths.
    use_gradient_checkpointing="unsloth",  

    random_state=3407,  # Random seed for reproducibility during training

    # Optional configurations for advanced techniques:
    use_rslora=False,  # Flag for whether to use Randomized Singular LoRA (RsLoRA)
    loftq_config=None,  # Configuration for LoFT-Q (LoRA for Fine-Tuning Quantized models)
)


In [ ]:
# Initialize the DPOTrainer (Direct Preference Optimization Trainer) to fine-tune the model
dpo_trainer = DPOTrainer(
    model=model,  # The model to be fine-tuned (already loaded and potentially quantized with LoRA)

    # Define training arguments for fine-tuning
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Number of samples processed per device in one forward/backward pass
        gradient_accumulation_steps=3,  # Accumulate gradients over multiple steps to simulate a larger batch size
        warmup_ratio=0.1,  # Fraction of total steps used for learning rate warmup
        num_train_epochs=1,  # Number of training epochs
        learning_rate=5e-6,  # Initial learning rate for the optimizer
        fp16=not torch.cuda.is_bf16_supported(),  # Use FP16 if BF16 is not supported
        bf16=torch.cuda.is_bf16_supported(),  # Use BF16 precision if supported by the GPU
        logging_steps=10,  # Log training progress every 10 steps
        optim="adamw_8bit",  # Use AdamW optimizer with 8-bit precision for memory efficiency
        weight_decay=0.001,  # Weight decay for regularization
        lr_scheduler_type="linear",  # Use a linear learning rate scheduler
        seed=42,  # Set random seed for reproducibility
        report_to="none",  # Disable reporting to external tools (e.g., WandB or TensorBoard)
        output_dir="outputs",  # Directory to save the model and training artifacts
    ),

    beta=0.1,  # Beta parameter for DPO; balances the loss for optimization
    train_dataset=dataset["train"],  # Training dataset
    eval_dataset=dataset["test"],  # Evaluation dataset
    tokenizer=tokenizer,  # Tokenizer corresponding to the model
    max_length=4096,  # Maximum token length for input sequences
    max_prompt_length=512,  # Maximum token length for prompts within input sequences
)

## Starting to finetune

In [ ]:
dpo_trainer.train() #start the training

## Saving Models

In [ ]:
model.save_pretrained("mistral-security-lora-trained")

In [ ]:
# to push to huggingface (optional)
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HuggigFace") # Fetching the Hugging Face token from the Kaggle Secret keys add on
login(token = hf_token) # Logging into Hugging Face Hub to access models and other resources
model.push_to_hub("mistral-security-lora-trained")

In [ ]:
from unsloth import FastLanguageModel
import transformers

In [ ]:
# Load the fine-tuned model and tokenizer from the specified directory
# The model is loaded with 4-bit quantization enabled for memory efficiency
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="mistral-security-lora-trained",  # Path to the saved fine-tuned model
    max_seq_length=4096,  # Set the maximum sequence length for processing inputs
    load_in_4bit=True,  # Enable 4-bit quantization to reduce memory usage
)

# Prepare the model for inference (set it to evaluation mode)
FastLanguageModel.for_inference(model)  # This configures the model for inference without further training

## Testing fine-tuned model

In [ ]:
def predict(input_prompt):
    # Define the conversation context for the model, including system and user roles
    messages = [
        {"role": "system", "content": "You are a helpful assistant that assists users to find the correct methods/approach for security within an organization."},
        {"role": "user", "content": input_prompt}  # The user's input prompt is added here
    ]

    # Format the input message into the appropriate structure for the model using the tokenizer
    text = tokenizer.apply_chat_template(
        messages,  # The structured message for the assistant
        tokenize=False,  # Don't tokenize yet, just prepare the chat template
        add_generation_prompt=True  # Add any additional generation prompt required by the model
    )

    # Tokenize the message text into model input format
    model_inputs = tokenizer([text], return_tensors="pt")

    # Generate model output by passing the tokenized input to the model
    # 'max_new_tokens' ensures that the model generates up to a maximum of 4096 new tokens
    generated_ids = model.generate(**model_inputs, max_new_tokens=4096,temperature=0.2)

    # Slice the generated sequence to remove the original input tokens (we only want the output)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

    # Decode the generated token IDs back into a human-readable string, skipping special tokens
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Return the generated response
    return response

In [ ]:
input_prompt = "What should be done when an employee leaves the organization?"
response = predict(input_prompt)
print(response)

In [ ]:
input_prompt = "Explain the concept of log management in the context of SIEM."
response = predict(input_prompt)
print(response)